Hi everyone! My name is Erick Ratamero, and I am a Systems Analyst at the Research IT department at the Jackson Laboratory, a research non-profit based in Bar Harbor, Maine, United States. Our group is called Imaging Solutions, and we provide hardware and software support for 70+ PIs and their groups on all their needs when it comes to imaging data.

First of all, thanks for the interest in this workshop! I will go through a very short presentation about ezomero, describing what it is and how it works, and then I will show you some concrete examples of ezomero in action. 

If you have not gone through our setup guide, now is a good time to pause this video and do so! We encourage everyone to follow along the workshop by actually writing the commands I will be showing and seeing their results on your own computer. To do that, you will need a Python environment with ezomero and access to an OMERO server. The setup guide goes into detail on how to get those things together.

(presentation goes here)

So before we start the actual workshop, I will go through a very simple example on how to retrieve an image from an OMERO server using omero-py and ezomero, just as a warm-up and a point of comparison. This is just a teaser for the things we'll be doing, and just to show that it's real easy to do with the scaffolding we've built.

Besides the two APIs we will show here, OMERO has a CLI interface that can do a lot of stuff - (almost?) all of it can also be done with the omero-py API, so it would be kinda redundant. But it's useful if you just want to write a bash script to automate something.

So first things first, I'm using a `config.py` file containing all the sensitive info here. In general I have this as a local file that won't be on the github repository, I make sure I have it accessible only for my system user and don't keep it saved between sessions (or at least keep it saved without the password). In this case I'm using a default user on a local server, so no reason not to show you. If you are using the docker-compose from this repository as your OMERO server, you can also use the file as is, otherwise make sure to alter it to point to your own OMERO server.

In [ ]:
from config import OMEROUSER, OMEROPASS, OMEROHOST, OMEROPORT, OMEROWEB

This is just some magic to make matplotlib plot things inline instead of popping up an extra window, and with specific figure format stuff.

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

Ok so first we will do the imports we need here and define which image ID we will show. You should import some data to your OMERO server before this step, so make sure to consult the setup guide and use OMERO.insight to upload some data. Then, you should choose the ID of one of the images available for you (use OMERO web in your browser to pick one). Other than that, we import the ezomero library and matplotlib to plot the images.

In [ ]:
import ezomero
import matplotlib.pyplot as plt
image_id = YOUR_ID_HERE

Here we just create a connection object using the contents from our config file. The empty string tells ezomero to use the default group for that user; if you don't put it there, no big deal. You will be asked to which group you want to connect and you can just press enter to use the default group.

In [ ]:
conn = ezomero.connect(OMEROUSER, OMEROPASS, "", host=OMEROHOST, port=OMEROPORT, secure=True)


Now we use the `conn` object as an argument in the `ezomero.get_image` function, with image ID and an optional argument specifiying we want XYZCT ordering for dimensions. We always get a 5d numpy array back from this helper function, so we have a look at the shape and then plot the first plane/timepoint/channel (we also need to transpose this to get it seen the same way as in OMERO.web due to what matplotlib consider X and Y axes to be.

In [ ]:
img, pix = ezomero.get_image(conn, image_id, xyzct=True)
print(pix.shape)
plt.imshow(pix[:,:,0,0,0].T)
conn.close()

Now we'll do the same thing using the omero-py API - you will see that there is a bit more work involved here, and the code is a bit less legible. We will import the `BlitzGateway` class form omero-py, create an object, then connect it. I'm printing the return code to make sure it worked (`True` is good). Then, we will use `getPlane` to get a single plane - note that if you wanted a full 5D array, you would need to retrieve the Z, C and T sizes of the image and then loop over all planes!

In [ ]:
from omero.gateway import BlitzGateway
conn = BlitzGateway(OMEROUSER, OMEROPASS, port=OMEROPORT, host=OMEROHOST)
ret = conn.connect()
print(ret)
image = conn.getObject('Image', image_id)
pixels = image.getPrimaryPixels()
plane = pixels.getPlane(0, 0, 0) 
plt.imshow(plane)